In [1]:
%matplotlib inline
import matplotlib.pylab as plt
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.utils as utils

In [2]:
conns = []
colmap_dicts = []
conns.append(db.SimpleDatabase('opsim3_61_sqlite.db', defaultTable='Summary', groupBy='expMJD',
                               defaultdbTables={'Summary': ['Summary', 'obsHistID']}))
colmap_dicts.append({'slewtime': 'slewTime', 'exptime': 'expTime', 'visittime': 'expTime', 'mjd':'expMJD'})




In [8]:
for conn,colmap_dict in zip(conns, colmap_dicts):
    bundleList = []
    sql = ''
    slicer = slicers.UniSlicer()
    metric = metrics.OpenShutterFractionMetric(slewTimeCol=colmap_dict['slewtime'],
                                               expTimeCol=colmap_dict['exptime'],
                                               visitTimeCol=colmap_dict['visittime'])
    bundle = metricBundles.MetricBundle(metric, slicer, sql)
    bundleList.append(bundle)
    metric = metrics.MedianMetric(col='normairmass')
    bundle = metricBundles.MetricBundle(metric, slicer, sql)
    bundleList.append(bundle)
    
    metric = metrics.MedianMetric(col=colmap_dict['slewtime'])
    bundle = metricBundles.MetricBundle(metric, slicer, sql)
    bundleList.append(bundle)
    
    bd = metricBundles.makeBundlesDictFromList(bundleList)
    group = metricBundles.MetricBundleGroup(bd, conn)
    group.runAll()

Querying database Summary with no constraint for columns ['airmass', 'fieldDec', 'expTime', 'slewTime'].
Found 2651588 visits
Running:  ['opsim_Median_slewTime_UNIS', 'opsim_OpenShutterFraction_UNIS', 'opsim_Median_normairmass_UNIS']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [9]:
bundleList[2].metricValues

masked_array(data = [4.65167],
             mask = [False],
       fill_value = -666.0)